In [12]:
class DCJsonData:
    def __init__(self, file_path):
        self.file_path = file_path
        self.json_data = self._load_json()

    def _load_json(self):
        try:
            f = open(self.file_path, "r")
        except FileNotFoundError:
            f = open(self.file_path, "w")
            f.write("{\"posts\":[]}")
            f.close()
            f = open(self.file_path, "r")
        return json.load(f)

    def save2txt(self):
        txt_path = '.'.join(self.file_path.split('.')[:-1] + ['txt'])
        with open(txt_path, 'w') as f:
            for postid in self.json_data.keys():
                post = self.json_data[postid]
                for tag in ["title", "content"]:
                    txt = post[tag]
                    if txt == '': continue
                    f.write(txt+'\n')
                for comment in post["comments"]:
                    f.write(comment["content"]+'\n')
                    if txt == '': continue
            f.close()

    def save(self, results):
        # json_data = {}
        # try:
        #     f = open(file_path, "r")
        # except FileNotFoundError:
        #     f = open(file_path, "w")
        #     f.write("{\"posts\":[]}")
        #     f.close()
        #     f = open(file_path, "r")

        # # with open(file_path, "r") as json_file:
        # json_data = json.load(f)

        for result in results:
            self.json_data['posts'].append(result)
        with open(self.file_path, 'w', encoding='utf-8') as outfile:
            json.dump(self.json_data, outfile, indent=4, ensure_ascii=False)
 

In [10]:
import json
item = json.load(open("dcinside_crawling.json", "r"))

In [ ]:
item.keys()

In [27]:
dcjson = DCJsonData('dcinside_crawling.json')

In [28]:
dcjson.save2txt()

## 코퍼스전처리
* rerefenced from : kcbert


In [ ]:
!pip install pip install soynlp emoji

In [12]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [8]:
cd /data/private/eta-crawler/src

/data/private/eta-crawler/src


In [25]:
retry_count = 0
for idx in range(7,8):
    # !sed -i '/^$/d' '{idx}.txt'
    with open(f'{idx}.txt', 'r') as f:
        with open(f'normalized/{idx}.txt', 'w') as wf:
            while True:
                txt = f.readline().strip()
                if txt == '': 
                    retry_count+=1
                    if retry_count >= 2: break
                    continue
                retry_count=0
                txt = clean(txt)
                wf.write(txt+'\n')
        

## 파일 다운로드

## Korpora (KcBERT) 다운로드

In [27]:
!pip install Korpora

Looking in indexes: https://pypi.org/simple, http://kakaobrain-pypi.dev.9rum.cc/


In [30]:
from Korpora import Korpora
Korpora.fetch('kcbert', root_dir='/data/private/')

[kcbert] download kcbert-train.tar.gzaa: 100%|██████████| 2.10G/2.10G [02:42<00:00, 12.9MB/s]    
[kcbert] download kcbert-train.tar.gzab: 100%|██████████| 2.10G/2.10G [02:42<00:00, 12.9MB/s]    
[kcbert] download kcbert-train.tar.gzac: 671MB [00:54, 12.3MB/s]                               


Unzip tar. It needs a few minutes ... done


## Huggingface Tokenizer 학습

In [ ]:
!pip install transformers

In [48]:
from transformers import ElectraConfig, ElectraModel, ElectraTokenizerFast
from tokenizers import BertWordPieceTokenizer

# paths = [str(x) for x in Path("./eo_data/").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = BertWordPieceTokenizer(
    vocab_file=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False
)
tokenizer.train(['kcbert/tokenizer_train.txt'], vocab_size=30000, limit_alphabet=4000)
tokenizer.save_model('kcbert', '30000')

['/data/private/kcbert/only-eta-vocab.txt']

In [10]:
# !cat /data/private/eta-crawler/src/normalized/*.txt /data/private/kcbert/kcbert.small.txt \
#     > /data/private/kcbert/tokenizer_train.txt

In [49]:
!mkdir -p /data/private/soongsil-electra/resource/raw
!cat  /data/private/eta-crawler/src/normalized/*.txt /data/private/kcbert/kcbert.txt \
    > /data/private/soongsil-electra/resource/raw/merged.txt
!du -hl /data/private/soongsil-electra/resource/raw/

14G	/data/private/soongsil-electra/resource/raw/


In [50]:
!shuf /data/private/soongsil-electra/resource/raw/merged.txt > /data/private/soongsil-electra/resource/raw/merged.shuf.txt
!rm /data/private/soongsil-electra/resource/raw/merged.txt

In [42]:
!mv /data/private/eta-crawler/src/normalized/5.txt /data/private/eta-crawler/src/normalized/5.raw

## Make Custom Config
config를 vocab사이즈를 맞추어 저장합니다.

In [1]:
size='small' # base, large

In [2]:
from transformers import ElectraConfig

disc = ElectraConfig.from_pretrained(f"google/electra-{size}-discriminator")
gen = ElectraConfig.from_pretrained(f"google/electra-{size}-generator")

In [7]:
disc.vocab_size = 16000
gen.vocab_size = 16000

In [8]:
disc

ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_size": 2,
  "vocab_size": 16000
}

In [9]:
gen

ElectraConfig {
  "architectures": [
    "ElectraForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "type_vocab_size": 2,
  "vocab_size": 16000
}

In [11]:
disc.save_pretrained('soongsil-small-discriminator')
gen.save_pretrained('soongsil-small-generator')